In [1]:
# Library import
import xml.etree.ElementTree as ET
import spacy
import string
import numpy as np
import json
import random
import pickle
# for auto-correct words in review text
from autocorrect import Speller
spell = Speller()

2022-04-27 12:26:28.826928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 12:26:28.840358: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 

In [2]:
xml_filepath = './dataset/ABSA16_Laptops_Train_SB1_v2.xml'
# create element tree object
tree = ET.parse(xml_filepath)

# get root element
root = tree.getroot()

In [3]:
data = []
for it1 in root.findall('Review'):
    for it2 in it1.findall('sentences'):
        for it3 in it2.findall('sentence'):
            for it4 in it3.findall('text'):
                review_text = spell(it4.text.lower())

            polarity_sum = 0
            for it4 in it3.findall('Opinions'):
                for it5 in it4.findall('Opinion'):
                    if it5.attrib['polarity'] == 'negative':
                        polarity_sum += -1
                    elif it5.attrib['polarity'] == 'neutral':
                        polarity_sum += 0
                    else:
                        polarity_sum += 1
                    
            # 0 negtaive, 1 neutral, 2 positive
            if polarity_sum == 0:
                polarity = 1  #neutral
            elif polarity_sum > 0:
                polarity = 2  #positive
            else:
                polarity = 0  #negative

            data.append([review_text,polarity])

In [4]:
print(len(data))

2500


In [5]:
# sm is small here, can use large here as lg here by downloading the large file
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")


# 200 dimensions vectors for the tokens. Giving 0's to padding token and generating randomly for oov tokens.
emmbedding_dimensions = 300
emmbed_dict = {}
emmbed_dict['pad'] = np.zeros(emmbedding_dimensions)

with open('../dataset/glove.6B/glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:],'float64')
        emmbed_dict[word]=vector

emmbed_dict['pad'] = np.zeros(emmbedding_dimensions)

vec = []
for _ in range(emmbedding_dimensions):
    vec.append(random.uniform(-1,1))

emmbed_dict['oov'] = np.array(vec)

In [6]:
# removed punctuations
train_data_vocab = set()  #also building training data vocab
train_data_vocab.add('pad')
train_data_vocab.add('oov')
max_sent_length = 0

for i in range(len(data)):
    sent = data[i][0]
    token_lst = []
    for it in nlp(sent):
        token = it.text
        str = ''
        for ch in token:
            if ch == "'" or ch not in string.punctuation:
                if ch == "'":
                    str += ' '
                str += ch
            else:
                str += ' '
                
        lst = str.split()
        for token in lst:
            if token in emmbed_dict:
                token_lst.append(token)
                train_data_vocab.add(token)
            else:
                token_lst.append('oov')

    max_sent_length = max(len(token_lst), max_sent_length)
    data[i][0] = token_lst


print('Maximum Sentence Length is: ',max_sent_length)

Maximum Sentence Length is:  75


In [7]:
# token to index for training data
token_to_index = {}
for cnt, token in enumerate(train_data_vocab):
    token_to_index[token] = cnt

In [8]:
# sentence level padding and converting to indices
for i in range(len(data)):
    sent_ind = []
    for j in range(max_sent_length):
        if j < len(data[i][0]):
            sent_ind.append(token_to_index[data[i][0][j]])
        else:
            sent_ind.append(token_to_index['pad'])
    data[i][0] = sent_ind

In [9]:
embedding_matrix = []
for token in train_data_vocab:
    embedding_matrix.append(emmbed_dict[token])

embedding_matrix = np.array(embedding_matrix)

In [10]:
# saving embedding matrix in numpy format
np.save('embedding_matrix_laptops_sentence_level',embedding_matrix)

In [11]:
dump_this_dict = {}
for i in range(len(data)):
    dump_this_dict[i] = data[i]

In [12]:
with open('train_laptops_sentence_level.json', 'wb') as f:
    pickle.dump(dump_this_dict, f)

In [13]:
with open('train_laptops_sentence_level.json', 'rb') as f:
    data = pickle.load(f)

In [14]:
# dumping token to index, so that we can create test data
with open('token_to_index_laptops_sentence_level.json', 'wb') as f:
    pickle.dump(token_to_index, f)

In [15]:
print(embedding_matrix.shape)

(3129, 300)


In [16]:
total = 0
pos = 0
neg = 0
neu = 0

for i in range(len(data)):
    if data[i][1] == 0:
        neg += 1
    elif data[i][1] == 1:
        neu += 1
    else:
        pos += 1

    total += 1

print(total, pos, neg, neu)

2500 1092 819 589
